In [182]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import pprint

In [183]:
df = pd.read_excel('Teste.xlsx',header=None) # O TESTE 3 É DA PÁGINA 282 DO HIBBELER lembrar de mudar "E" e "I" com base nos testes
df
# número de nós
# ID / x / y / restrição em x / restrição em y / restrição momento
# número de materiais
# ID / módulo de elasticidade
# número de barras
# ID / nó inicial / nó final / secão x / seção y / E
# número de cargas em nós
# nó / valor da carga x / valor da carga y / valor carga momento
# número de cargas em barras
# barra / inicio da carga(posição) / fim da carga(posição) / valor da carga no inicio / valor da carga no fim

,0,1,2,3,4,5
0,3,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,1.0,1.0,1.0
2,2,0.0,4.0,0.0,0.0,0.0
3,3,6.0,4.0,1.0,1.0,1.0
4,1,NaN,NaN,NaN,NaN,NaN
5,1,1.0,NaN,NaN,NaN,NaN
6,2,NaN,NaN,NaN,NaN,NaN
7,1,1.0,2.0,1.0,1.0,1.0
8,2,2.0,3.0,1.0,1.0,1.0
9,2,NaN,NaN,NaN,NaN,NaN


In [184]:


class No:
    def __init__(self, x, y, Rx, Ry, Rm):

        self.x = x          #posição x do nó
        self.y = y          #posição y do nó
        self.Rx = Rx        #apoio na direção x (0 sem apoio, 1 com apoio)
        self.Ry = Ry        #apoio na direção y (0 sem apoio, 1 com apoio)
        self.Rm = Rm        #apoio em relação ao momento (0 sem apoio, 1 com apoio)
        self.carga = [0,0,0]

    def __str__(self):
        return f'x: {self.x} y: {self.y} Rx: {self.Rx} Ry: {self.Ry} Rm: {self.Rm} carga: {self.carga}'

    def adicionarcarga(self,carga):
      self.carga[0] = self.carga[0] + carga.forcax
      self.carga[1] = self.carga[1] + carga.forcay
      self.carga[2] = self.carga[2] + carga.forcamomento







class Material:
    def __init__(self, E):
      self.E = E            #módulo de elasticidade








class Barra:
    def __init__(self, no1, no2, secaox, secaoy, E):

      self.no1 = no1
      self.no2 = no2
      self.secaox = secaox
      self.secaoy = secaoy
      self.L = calculacomprimento(no1,no2)
      self.I = 1#calculainercia(secaox,secaoy)
      self.Area = calculaarea(secaox,secaoy)
      self.E = E

      self.anguloeixox = calculaangulo(self)

    def __str__(self):
        return f'no1: {self.no1} \n\n no2: {self.no2}\n\n secaox: {self.secaox} secaoy: {self.secaoy} Comprimento: {self.L} I: {self.I} Area: {self.Area}'



        


class Carga:
    def __init__(self, forcax, forcay, forcamomento):
      self.forcax = forcax
      self.forcay = forcay
      self.forcamomento = forcamomento
  # é necessário atualizar para possibilitar a utilização de outros tipos de carga,
                             # como carga distribuída, ou uma carag não aplicada em um nó







def calculaarea(secaox,secaoy):
  return secaox*secaoy



def calculainercia(secaox,secaoy):
  return secaox*(secaoy**3)/12



def calculacomprimento(no1,no2):

  return np.sqrt(pow(abs(no1.x - no2.x),2) + pow(abs(no1.y - no2.y),2))

def calculaangulo(barra):

  if (barra.no2.x - barra.no1.x) == 0:
    return np.pi/2
  else:
    return np.arctan((barra.no2.y - barra.no1.y)/(barra.no2.x - barra.no1.x)) #calcula o angulo da barra em relação ao eixo x

In [185]:
class estrutura:
	def __init__(self, df):
		self.nos = {}
		self.barras = {}
		self.materiais = {}

		# leitura dos nós
		pos = 0
		num_nos = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_nos+1):
			self.nos[df.iloc[pos, 0]] = No(df.iloc[pos, 1], df.iloc[pos, 2], df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5])
			pos += 1

		# leitura dos materiais
		num_materiais = df.iloc[pos, 0]
		pos = pos + 1
		self.materiais[df.iloc[pos, 0]] = Material(df.iloc[pos, 1])
		pos += 1
		


		# leitura das barras
		num_barras = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_barras+1):
			self.barras[df.iloc[pos, 0]] = Barra(self.nos[df.iloc[pos, 1]], self.nos[df.iloc[pos, 2]], df.iloc[pos, 3], df.iloc[pos, 4],df.iloc[pos, 5])
			pos += 1

		# leitura das cargas nos nós
		num_cargas = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_cargas+1):
			x = Carga(df.iloc[pos, 1], df.iloc[pos, 2], df.iloc[pos, 3])	
			
			self.nos[df.iloc[pos, 0]].adicionarcarga(x)
			pos += 1

		# leitura das cargas nas barras
		num_cargas = df.iloc[pos, 0]
		print(num_cargas)
		pos = pos + 1

		for i in range(1, num_cargas+1):

			if df.iloc[pos, 1] > self.barras[df.iloc[pos, 0]].L:
				print('erro você está tentando aplicar uma carga em um ponto que não existe')
				break
			if df.iloc[pos, 2] > self.barras[df.iloc[pos, 0]].L:
				print('erro você está tentando aplicar uma carga em um ponto que não existe')
				break

			#Casos de cargas


			
			#Carga aplicada em um ponto da barra
			if df.iloc[pos, 1] == df.iloc[pos, 2]:

				if df.iloc[pos, 3] != df.iloc[pos, 4]:
					print('você está tentando aplicar uma carga variável em um ponto da barra')
					break
					
				no1_aux = self.barras[df.iloc[pos, 0]].no1
				no2_aux = self.barras[df.iloc[pos, 0]].no2
				
				a = df.iloc[pos, 1]
				b = self.barras[df.iloc[pos, 0]].L - df.iloc[pos, 1]

				cargayno1 = df.iloc[pos, 3] * b**3 * (3*a + b)/(self.barras[df.iloc[pos, 0]].L**3)
				cargayno2 = df.iloc[pos, 3] * a**3 * (a + 3*b)/(self.barras[df.iloc[pos, 0]].L**3)

				cargamomentono1 =   df.iloc[pos, 3] * a * b**2 / (self.barras[df.iloc[pos, 0]].L**2)
				cargamomentono2 = - df.iloc[pos, 3] * a**2 * b / (self.barras[df.iloc[pos, 0]].L**2) 
		
				no1_aux.adicionarcarga(Carga(0,cargayno1,cargamomentono1))
				no2_aux.adicionarcarga(Carga(0,cargayno2,cargamomentono2))


			#Carga aplicada em um intervalo da barra
			elif df.iloc[pos, 1] != df.iloc[pos, 2]:

				no1_aux = self.barras[df.iloc[pos, 0]].no1
				no2_aux = self.barras[df.iloc[pos, 0]].no2

				if abs(df.iloc[pos, 3]) == abs(df.iloc[pos, 4]):
					print('carga distribuída constante')

					cargayno1 = df.iloc[pos, 3] * self.barras[df.iloc[pos, 0]].L / 2
					cargayno2 = df.iloc[pos, 4] * self.barras[df.iloc[pos, 0]].L / 2

					cargamomentono1 = df.iloc[pos, 3] * self.barras[df.iloc[pos, 0]].L**2 / 12
					cargamomentono2 = -df.iloc[pos, 4] * self.barras[df.iloc[pos, 0]].L**2 / 12
				
				else:
					print('carga distribuída variavel')

					if abs(df.iloc[pos, 3]) > abs(df.iloc[pos, 4]):
						cargayno1 = 3 * df.iloc[pos, 4] * self.barras[df.iloc[pos, 0]].L / 20
						cargayno2 = 7 * df.iloc[pos, 4] * self.barras[df.iloc[pos, 0]].L / 20

						cargamomentono1 =   df.iloc[pos, 4] * self.barras[df.iloc[pos, 0]].L**2 / 30
						cargamomentono2 = - df.iloc[pos, 4] * self.barras[df.iloc[pos, 0]].L**2 / 20

					elif abs(df.iloc[pos, 3]) < abs(df.iloc[pos, 4]):
						cargayno1 = 7 * df.iloc[pos, 3] * self.barras[df.iloc[pos, 0]].L / 20
						cargayno2 = 3 * df.iloc[pos, 3] * self.barras[df.iloc[pos, 0]].L / 20

						cargamomentono1 =   df.iloc[pos, 3] * self.barras[df.iloc[pos, 0]].L**2 / 20
						cargamomentono2 = - df.iloc[pos, 3] * self.barras[df.iloc[pos, 0]].L**2 / 30


				


				no1_aux.adicionarcarga(Carga(0,cargayno1,cargamomentono1))
				no2_aux.adicionarcarga(Carga(0,cargayno2,cargamomentono2))
			print ('Barra: ', df.iloc[pos, 0],' yno1: ',cargayno1, ' yno2: ', cargayno2, ' momento no1: ', cargamomentono1, ' momento no2: ', cargamomentono2, '\n')
			pos += 1


			

	def calculaliberdade(Nos):
		grauliberdade = 0
		for no in Nos:
			if no.Rx == 1:
				grauliberdade += 1
			if no.Ry == 1:
				grauliberdade += 1
			if no.Rm == 1:
				grauliberdade += 1

		return grauliberdade - 3

	def __str__(self):
		st = 'Estrutura'
		st += '\nNos:'
		for no in self.nos:
			st += '\n' + str(no) + ': ' + str(self.nos[no])
		st += '\nBarras:'
		for barra in self.barras:
			st += '\n' + str(barra) + ': ' + str(self.barras[barra])
			st += '\n'
		return st



In [186]:
struct = estrutura(df)

for i in struct.nos:
    print(struct.nos[i].carga)
    

print(struct)

0
[0, 0, 0]
[10.0, -6.0, 0.0]
[0, 0, 0]
Estrutura
Nos:
1: x: 0.0 y: 0.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0]
2: x: 0.0 y: 4.0 Rx: 0.0 Ry: 0.0 Rm: 0.0 carga: [10.0, -6.0, 0.0]
3: x: 6.0 y: 4.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0]
Barras:
1: no1: x: 0.0 y: 0.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0] 

 no2: x: 0.0 y: 4.0 Rx: 0.0 Ry: 0.0 Rm: 0.0 carga: [10.0, -6.0, 0.0]

 secaox: 1.0 secaoy: 1.0 Comprimento: 4.0 I: 1 Area: 1.0

2: no1: x: 0.0 y: 4.0 Rx: 0.0 Ry: 0.0 Rm: 0.0 carga: [10.0, -6.0, 0.0] 

 no2: x: 6.0 y: 4.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0]

 secaox: 1.0 secaoy: 1.0 Comprimento: 6.0 I: 1 Area: 1.0



In [187]:
from sympy import symbols
from sympy.abc import E,A,L,I,theta
sp.init_printing(use_unicode=True, num_columns=200)

#matriz de rigidez de uma barra
K = sp.Matrix([[E*A/L       ,0            ,0           ,-E*A/L     ,0            ,0          ],
               [0           ,12*E*I/L**3  ,6*E*I/L**2  ,0          ,-12*E*I/L**3 ,6*E*I/L**2 ],
               [0           ,6*E*I/L**2   ,4*E*I/L     ,0          ,-6*E*I/L**2  ,2*E*I/L    ],
               [-E*A/L      ,0            ,0           ,E*A/L      ,0            ,0          ],
               [0           ,-12*E*I/L**3 ,-6*E*I/L**2 ,0          ,12*E*I/L**3  ,-6*E*I/L**2],
               [0           ,6*E*I/L**2   ,2*E*I/L     ,0          ,-6*E*I/L**2  ,4*E*I/L    ]])

In [188]:
print(struct.barras[1].anguloeixox)

1.5707963267948966


In [189]:


def calculamatrizlocal(barra):
    restricoes1 = [barra.no1.Rx,barra.no1.Ry,barra.no1.Rm]
    restricoes2 = [barra.no2.Rx,barra.no2.Ry,barra.no2.Rm]

    K_local = K.subs([(E,barra.E),(A,barra.Area),(L,barra.L),(I,barra.I),(theta,barra.anguloeixox)])
    #rotaciona a matriz de rigidez local
    T = sp.Matrix([[sp.cos(theta) ,-sp.sin(theta) ,0             ,0             ,0             ,0],
                   [sp.sin(theta),sp.cos(theta) ,0             ,0             ,0             ,0],
                   [0             ,0             ,1             ,0             ,0             ,0],
                   [0             ,0             ,0             ,sp.cos(theta) ,-sp.sin(theta) ,0],
                   [0             ,0             ,0             ,sp.sin(theta),sp.cos(theta) ,0],
                   [0             ,0             ,0             ,0             ,0             ,1]])
    rotacao = T.subs([(theta,barra.anguloeixox)]).copy()

    #Faz a rotação da matriz de rigidez local
    K_local = rotacao.transpose()*K_local*rotacao
    


    for i in range(0,3):
        if restricoes1[i] == 1:
            for j in range(0,6):
                K_local[i,j] = 0
                K_local[j,i] = 0
        if restricoes2[i] == 1:
            for j in range(0,6):
                K_local[i+3,j] = 0
                K_local[j,i+3] = 0

    # Defina um limite para considerar um valor como zero
    limite = 1e-15

    # Substitue valores próximos de zero por zero
    K_local = K_local.applyfunc(lambda x: 0 if abs(x) < limite else x)
    
    pprint(K_local)

    #aplica as restrições
    return K_local
    
    

In [190]:
K_local = {}
#Recupera as matrizes locais
for i in struct.barras:
    K_local[i] = calculamatrizlocal(struct.barras[i])

dimensao = 3 + 3 * len(struct.barras)

#Cria a matriz global e a matriz de forças
K_global = sp.zeros(dimensao,dimensao)
F = sp.zeros(dimensao,1)

#Soma dos vetores de rigidez locais
for i, barra in enumerate(struct.barras):
    for j in range(0,6):
        for k in range(0,6):
            K_global[j + 3  *i, k + 3 * i] = K_global[j + 3 * i, k + 3 * i] + K_local[barra][j,k]
print('Matriz de rigidez global')
pprint(K_global)
print('\n')
#Criaçãpo do vetor de forças
for i, no in enumerate(struct.nos):
    F[3 * i] = -struct.nos[no].carga[0]
    F[3 * i + 1] = -struct.nos[no].carga[1]
    F[3 * i + 2] = -struct.nos[no].carga[2]
    
print('Vetor de forças')
pprint(F)

⎡0  0  0    0      0      0   ⎤
⎢                             ⎥
⎢0  0  0    0      0      0   ⎥
⎢                             ⎥
⎢0  0  0    0      0      0   ⎥
⎢                             ⎥
⎢0  0  0  0.1875   0    -0.375⎥
⎢                             ⎥
⎢0  0  0    0     0.25    0   ⎥
⎢                             ⎥
⎣0  0  0  -0.375   0     1.0  ⎦
⎡0.166666666666667          0                   0          0  0  0⎤
⎢                                                                 ⎥
⎢        0          0.0555555555555556  0.166666666666667  0  0  0⎥
⎢                                                                 ⎥
⎢        0          0.166666666666667   0.666666666666667  0  0  0⎥
⎢                                                                 ⎥
⎢        0                  0                   0          0  0  0⎥
⎢                                                                 ⎥
⎢        0                  0                   0          0  0  0⎥
⎢                                   

In [191]:
def deletar_colunas_e_linhas_vazias(matrix,vector):
    # Converte a matriz para numpy
    matrix_np = np.array(matrix.tolist())

    # acha as linhas e colunas vazias
    linhas_vazias = np.where(np.all(matrix_np == 0, axis=1))[0]
    colunas_vazias = np.where(np.all(matrix_np == 0, axis=0))[0]


    # Deleta as linhas e colunas vazias
    matrix_np = np.delete(matrix_np, linhas_vazias, axis=0)
    matrix_np = np.delete(matrix_np, colunas_vazias, axis=1)

    vector = np.delete(vector, linhas_vazias, axis=0)

    matrix = sp.Matrix(matrix_np)

    return matrix, vector    

In [192]:
K_global, F = deletar_colunas_e_linhas_vazias(K_global,F)

pprint(K_global)
print('\n')

print('X: ',F[0])
print('Y: ',F[1])


⎡0.354166666666667          0               -0.375      ⎤
⎢                                                       ⎥
⎢        0          0.305555555555556  0.166666666666667⎥
⎢                                                       ⎥
⎣     -0.375        0.166666666666667  1.66666666666667 ⎦


X:  [-10.0000000000000]
Y:  [6.00000000000000]


In [193]:

# Convert K_global to a float64 array
K_global_float = np.array(K_global, dtype=np.float64)
F = np.array(F, dtype=np.float64)

# Solve the linear system
x = np.linalg.solve(K_global_float, F)

# Print the solution
for i, f in enumerate(x):
    print(f'd{i} = {f}')


d0 = [-40.68657845]
d1 = [26.05066163]
d2 = [-11.75954631]
